In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
ask_men_url = 'https://www.reddit.com/r/askmen.json'
ask_women_url = 'https://www.reddit.com/r/askwomen.json'

In [3]:
headers = {'User-agent': 'Aidan'}

In [4]:
ask_men_res = requests.get(ask_men_url, headers=headers)
ask_women_res = requests.get(ask_women_url, headers=headers)

In [5]:
ask_men_res.status_code

200

In [6]:
ask_women_res.status_code

200

In [7]:
men_json = ask_men_res.json()
women_json = ask_women_res.json()

In [8]:
men_json.keys()

dict_keys(['kind', 'data'])

In [9]:
men_json['data'].keys()

dict_keys(['modhash', 'dist', 'children', 'after', 'before'])

In [10]:
len(men_json['data']['children'])

27

In [11]:
men_json['data']['after']

't3_abk4lr'

In [12]:
[post['data']['name'] for post in men_json['data']['children']]

['t3_aa9wmy',
 't3_abcf7l',
 't3_abjdqk',
 't3_abe82i',
 't3_abk1zu',
 't3_abil9q',
 't3_ablpxh',
 't3_abm69x',
 't3_abh87l',
 't3_abgu01',
 't3_abiyxz',
 't3_abmv80',
 't3_abhp0w',
 't3_abmgqc',
 't3_abkgpg',
 't3_abm0db',
 't3_abk8n6',
 't3_abi673',
 't3_abfbkw',
 't3_ab5oww',
 't3_abko36',
 't3_abj8p3',
 't3_abhzlt',
 't3_abmero',
 't3_abiv1f',
 't3_abn00i',
 't3_abk4lr']

I used to pushshift.io API in order to obtain the .jsons that I needed as it was quick and efficient.  Below I am testing out how to loop through the json and assing key value pairs to be put into a dictionary. 

In [13]:
params = {'size':500,'subreddit':'askmen','after':'1d','before':'0d'}

In [14]:
url = 'https://api.pushshift.io/reddit/search/submission/'
response = requests.get(url, params=params)
print('Status Code: ', response.status_code)  

Status Code:  200


In [15]:
men_json = response.json()

In [16]:
men_posts = []
for i in men_json['data']:
    post = {}
    post['title'] = i['title']
    post['body'] = i['selftext']
    post['author'] = i['author']
    post['subreddit'] = i['subreddit']
    post['id'] = i['id']
    men_posts.append(post)

In [17]:
df = pd.DataFrame(men_posts)

In [18]:
df.head()

,author,body,id,subreddit,title
0,kinspace,I'm curious as to what draws in the average pe...,abcly6,AskMen,For anyone that's paid for pics/premium Snapch...
1,Petunia808,[removed],abcm7j,AskMen,do you think current singers are bad?
2,inthedarkend,,abcmky,AskMen,How do you handle food/cooking/shopping once y...
3,RisenFlames,Just for context this is my first time using a...,abcmsv,AskMen,Why do men ghost then message again after awhile?
4,Captain_Stottlemeyer,,abcqpe,AskMen,How long did it take for you to take the next ...


In [19]:
params = {'size':500,'subreddit':'askmen','after':'4d','before':'3d'}
url = 'https://api.pushshift.io/reddit/search/submission/'
response = requests.get(url, params=params)
men_json = response.json()
men_posts = []
for k, v in enumerate(men_json['data']):
    post = {}
    print(k, v['selftext'])
    post['title'] = v['title']
    post['body'] = v['selftext']
    post['author'] = v['author']
    post['subreddit'] = v['subreddit']
    post['id'] = v['id']
    men_posts.append(post)
df = pd.DataFrame(men_posts)
df.head()

0 [removed]
1 [removed]
2 [removed]
3 I know this isnt the place but, I'm in the 8th grade I have this one friend that was normal at first but now jokes about shooting up you know what ,when he was normal i told him alot of things that would ruin my life at school like who i am tryin to get with and what not. Now this sounds easy to solve, except for he goes to the same school as me ,same classes ,same neighborhood, same way home ,... same friends.
He's disturbing ,weird ,and annoying. He throws shade on my family. And everytime I say shit bout his family he dont like it and talks bout respect like he's some god. He tortures sims on sims 4 ,and more on him being disturbing , every opportunity he has he talks about who died recently how he's surprised that some celebrities have died, I've gone on his youtube history before all it is ,is stuff like the Columbine shooting and all like before they were dead episodes, please help me end this friendship.
4 
5 [removed]
6  So i don't know wha

,author,body,id,subreddit,title
0,JustCardz,[removed],aafnvz,AskMen,Does anyone else get bored of sex too ?
1,SaltyIslander,[removed],aafnwi,AskMen,"I feel I lost my sexual drive at a young age, ..."
2,chrysanthemym,[removed],aafpkv,AskMen,[Serious] Does going over their house assume s...
3,100plusdubs,"I know this isnt the place but, I'm in the 8th...",aafqdj,AskMen,How to end a friendship with someone who knows...
4,squigleywrites,,aafqip,AskMen,"What does masculinity mean to you, personally?"


One thing I noticed when collecting the posts from reddit was that many of the posts were showing up with blank body text, or [deleted] or [removed] in the body.  Since I wanted to use both the title and the body of the posts in my classification model, I chose to simply not use the posts with no body text, and as you can see later I removed them from the dataframe that I run a function to continuously loop through reddit to get a certain number of posts

In [20]:
params = {'size':500,'subreddit':'askmen','after':'4d','before':'3d'}
url = 'https://api.pushshift.io/reddit/search/submission/'
response = requests.get(url, params=params)
men_json = response.json()
men_json['data'][99]

{'author': 'satansbootyhole34',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_2dsktv8r',
 'author_patreon_flair': False,
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1546057692,
 'domain': 'self.AskMen',
 'full_link': 'https://www.reddit.com/r/AskMen/comments/aahzoj/why_do_you_reject_some_women/',
 'gildings': {'gid_1': 0, 'gid_2': 0, 'gid_3': 0},
 'id': 'aahzoj',
 'is_crosspostable': False,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': False,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 2,
 'num_crossposts': 0,
 'over_18': False,
 'parent_whitelist_status': 'all_ads',
 'permalink': '/r/AskMen/comments/aahzoj/why_do_y

In [21]:
df['no_body'] = df['body'].map(lambda x: 1 if (x=='') | (x=='[removed]') else 0)

In [22]:
df[df['no_body']==1].head(20)

,author,body,id,subreddit,title,no_body
0,JustCardz,[removed],aafnvz,AskMen,Does anyone else get bored of sex too ?,1
1,SaltyIslander,[removed],aafnwi,AskMen,"I feel I lost my sexual drive at a young age, ...",1
2,chrysanthemym,[removed],aafpkv,AskMen,[Serious] Does going over their house assume s...,1
4,squigleywrites,,aafqip,AskMen,"What does masculinity mean to you, personally?",1
5,hahanooneknowsimhere,[removed],aafqju,AskMen,What's a can't-miss action movie I need to watch?,1
7,evanduong,,aafr40,AskMen,When's it okay to cry/sob?,1
8,dontfree69,[removed],aafrvv,AskMen,Is it mandatory to lift up the toilet seat whe...,1
10,ActuallyyesNo,[removed],aafudt,AskMen,Not interested or simply shy?,1
11,_MddM_,[removed],aafxsv,AskMen,Not interested or simply shy?,1
12,nariman9898,[removed],aafxyb,AskMen,There’s this girl I like a lot for 3 years alr...,1


In [23]:
df['no_body'].sum()

224

In [24]:
df.shape

(347, 6)

In [25]:
df.head(10)

,author,body,id,subreddit,title,no_body
0,JustCardz,[removed],aafnvz,AskMen,Does anyone else get bored of sex too ?,1
1,SaltyIslander,[removed],aafnwi,AskMen,"I feel I lost my sexual drive at a young age, ...",1
2,chrysanthemym,[removed],aafpkv,AskMen,[Serious] Does going over their house assume s...,1
3,100plusdubs,"I know this isnt the place but, I'm in the 8th...",aafqdj,AskMen,How to end a friendship with someone who knows...,0
4,squigleywrites,,aafqip,AskMen,"What does masculinity mean to you, personally?",1
5,hahanooneknowsimhere,[removed],aafqju,AskMen,What's a can't-miss action movie I need to watch?,1
6,JustCardz,So i don't know what to think of it so i just...,aafr16,AskMen,How come I am bored during sex ?,0
7,evanduong,,aafr40,AskMen,When's it okay to cry/sob?,1
8,dontfree69,[removed],aafrvv,AskMen,Is it mandatory to lift up the toilet seat whe...,1
9,COW321,"I'm 17 atm, my life was a rollecoaster tbh.\nI...",aaft97,AskMen,How do I know I'm mature enough for a relation...,0


In [26]:
df = df.drop(df[df['no_body']==1].index)

In [27]:
df.shape

(123, 6)

In [28]:
def pushshift_scrape_clean(subreddit, after, before, size=500):
    params = {'size':size,'subreddit': subreddit,'after': after,'before': before}
    url = 'https://api.pushshift.io/reddit/search/submission/'
    response = requests.get(url, params=params)
    the_json = response.json()
    
    posts = []
    for i in the_json['data']:
        if 'selftext' in i.keys():
            post = {}
            post['title'] = i['title']
            post['body'] = i['selftext']
            post['author'] = i['author']
            post['subreddit'] = i['subreddit']
            post['id'] = i['id']
            posts.append(post)
        else:
            pass
    
    df = pd.DataFrame(posts)
    df['no_body'] = df['body'].map(lambda x: 1 if (x=='') | (x=='[removed]') | (x=='[deleted]') else 0)
    df = df.drop(df[df['no_body']==1].index)
    df = df.drop('no_body', axis=1)
    return df

In [29]:
men_1 = pushshift_scrape_clean(subreddit='askmen', after='1d', before='0d', size=500)
print(men_1.shape)
men_1.head()

(133, 5)


,author,body,id,subreddit,title
0,kinspace,I'm curious as to what draws in the average pe...,abcly6,AskMen,For anyone that's paid for pics/premium Snapch...
3,RisenFlames,Just for context this is my first time using a...,abcmsv,AskMen,Why do men ghost then message again after awhile?
6,MediocrePenisNumber6,I'm not trying to offend anyone I just want to...,abcrua,AskMen,Guys who where the school pretty boy and guys ...
11,Greatpocketlintking,GF found out I use mine on my whole body and s...,abcxuj,AskMen,Men of reddit who use a loofah in the shower; ...
12,Cheetobear6891,Or is it time to go joe rogan on this shit?,abd0u0,AskMen,What has been your experience with hair loss t...


In [30]:
men_2 = pushshift_scrape_clean(subreddit='askmen', after= '2d', before='1d', size=500)
print(men_2.shape)
men_2.head()

(120, 5)


,author,body,id,subreddit,title
0,jdog1067,Let me preface by explaining my situation. I’v...,ab16dg,AskMen,Men who had adult girlfriends with strict pare...
3,throwaway64849294772,Hi! Girl here! I'm hoping I don't make this to...,ab1b13,AskMen,FWB Help? Opinions? Advice needed.
8,tokillatequila,How long did it take you to get over performan...,ab1nbp,AskMen,Hi guys
9,loveunicorn16,"Me and my ex-boyfriend broke up in March, he b...",ab1pmm,AskMen,ex boyfriend help
14,VelvetThunderstorm,Or what makes your current SO the best?,ab1v9x,AskMen,Why isn't your SO the best sex you've ever had?


In [31]:
men_3 = pushshift_scrape_clean(subreddit='askmen', after= '3d', before='2d', size=500)
print(men_3.shape)
men_3.head()

(100, 5)


,author,body,id,subreddit,title
2,ElizzaBum,So my boyfriend has asked for a blowjob the la...,aaqcql,AskMen,What are some reasons that you can't cum durin...
5,everythangspeachie,I just looked at myself and noticed actual wri...,aaqdw8,AskMen,(M25) man i just looked at myself in the mirro...
12,shway7,Girls who are around 5’10-6’1. Do you find the...,aaqjpl,AskMen,What’s your opinion on tall girls?
20,Urisk,Was there ever something you changed about yo...,aaqwda,AskMen,What is one change you've made in your life th...
22,Bnooc,Hi there :)\n\nAs the title says I wonder wha...,aaqww8,AskMen,What product(s) you discovered this year that ...


In [32]:
def scrape_loop(subreddit):
    df = pd.DataFrame()
    b = 0
    a = 1
    while len(df) < 25000:
        temp_df = pushshift_scrape_clean(subreddit=subreddit, after=(str(a)+'d'), before=(str(b)+'d'), size=500)
        print(b)
        print(a)
        df = pd.concat([df, temp_df])
        b += 1
        a += 1
    return df

In [33]:
men_df = scrape_loop('askmen')

0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151
152
15

In [34]:
len(men_df['id'].unique())

25022

In [36]:
women_df = scrape_loop('askwomen')

0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151
152
15

In [37]:
len(women_df['id'].unique())

25000

In [38]:
all_posts = pd.concat([men_df, women_df])

In [39]:
all_posts.to_csv('./all_posts_scraped_clean_50')

As seen above, I created a function that to do all of the web scraping and cleaning for 500 posts at a time using the pushshift.io API called pushshift_scrape_clean().  In this function as I mentioned before I created a dummy that flags posts that do not have body text, and removed them from the dataframe that I created.  

I then created another function in order to continuously scrape the reddit thread using my first function called scrape_loop().  With this function I allowed parameters to be passed in order to select the reddit thread to be scraped, and the starting data from which to begin scraping posts.  In the function there is a while loop that I set in order to stop the scraping once a certain number of posts had been reached.  I started this off at just 1000 from each thread, but I came back here once I began modeling and increased the number of posts I scraped in order to improve my model.  I did this a few times, saving CSVs of 10,000, 30,000, and 50,000 posts. 